In [1]:
import pandas as pd
import collections
import etl_idesp

In [2]:
df_idesp = etl_idesp.etl()

In [3]:
df_idesp.head(3)

,id_gestao,id_escola,ano_gestao,nota_ano,variacao
0,esc-10005-ano-2014,10005,2014,3.48,10
1,esc-10005-ano-2015,10005,2015,3.78,8
2,esc-10005-ano-2016,10005,2016,3.26,-13


In [4]:
len(set(df_idesp.id_escola))

5457

In [5]:
esc = {}
esc['id_escola'] = []
esc['nota'] = []
for nome, dados in df_idesp.groupby('id_escola'):
    esc['id_escola'].append(nome)
    esc['nota'].append(dados.nota_ano.mean())
esc = pd.DataFrame(esc)

# CLUSTER
- CD_CLUSTERS e o id
- CD_DIRETORIA ha 91 diretorias
- Numero do cluster: 9
- NR_CICLO: 3
- CD_GRUPO_CLUSTER

In [6]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados\\cluster_de_escolas_similares\\12_MMR_Clusters.csv'
cluster = pd.read_csv(arquivo, sep=';')
cluster.CD_ESCOLA = cluster.CD_ESCOLA.astype(int)
cluster = cluster.rename({'CD_ESCOLA': 'id_escola'}, axis=1)

In [7]:
clu = {}
clu['id_escola'] = []
clu['nr_cluster'] = []
for nome, dados in cluster.groupby('id_escola'):
    clu['id_escola'].append(nome)
    clu['nr_cluster'].append(dados.NR_CLUSTER.max())  


In [8]:
clu = pd.DataFrame(clu)

In [9]:
esc_cl = pd.merge(esc, clu, on='id_escola',how='left')

In [10]:
esc_cl.loc[:,['nota','nr_cluster']].corr()

,nota,nr_cluster
nota,1.000000,-0.220918
nr_cluster,-0.220918,1.000000


In [11]:
esc_cl.nr_cluster.describe()

count    2134.000000
mean        4.126054
std         2.022273
min         1.000000
25%         2.000000
50%         4.000000
75%         6.000000
max         9.000000
Name: nr_cluster, dtype: float64

In [30]:
esc_cl[esc_cl.nr_cluster.isin([1,2,3,4])].nota.mean()

4.40099133369823

In [39]:
esc_cl[esc_cl.nr_cluster.isin([1,2,3,4])].nota.count()

1218

In [31]:
esc_cl[esc_cl.nr_cluster.isin([5,9])].nota.mean()

4.121484203296703

In [40]:
esc_cl[esc_cl.nr_cluster.isin([5,9])].nota.count()

364

In [32]:
esc_cl[esc_cl.nr_cluster.isin([6,7,8])].nota.mean()

3.5821358192431565

In [41]:
esc_cl[esc_cl.nr_cluster.isin([6,7,8])].nota.count()

552

In [33]:
esc_cl[esc_cl.nr_cluster == 0].nota.mean()

2.776062744507975

In [38]:
esc_cl[esc_cl.nr_cluster == 0].count()

id_escola     3323
nota          3323
nr_cluster    3323
dtype: int64

# NUMERO DE CLASSES

In [30]:
arquivo = 'dados\matriculas\/10_Escolas_Classes_Qtde_Alunos.csv'
classes = pd.read_csv(arquivo, sep=';')

In [32]:
classes.COD_ESC = classes.COD_ESC.astype(int)

In [34]:
classes = classes.rename({'COD_ESC': 'id_escola'}, axis=1)

In [ ]:
result = pd.merge(esc_cl, )

In [36]:
classes.head(10)

,ANO,NOMEDEP,DE,MUN,DISTR,id_escola,NOMESC,TIPOESC,TIPOESC_DESC,SITUACAO,CLASSE,TIPOCLASSE,TIPOCLASSE_DESC,GRAU,TipoEnsino,SERIE,QTDE_ALUNOS,CODVINC
0,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,182570713,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,8,21,NaN
1,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,182570762,0,CLASSE COMUM,2,ENSINO MEDIO,2,37,NaN
2,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,186061859,1,CLASSE MULTISSERIADA,5,EJA ENSINO MÉDIO,10,25,NaN
3,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570440,0,CLASSE COMUM,2,ENSINO MEDIO,2,26,NaN
4,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570465,0,CLASSE COMUM,2,ENSINO MEDIO,3,33,NaN
5,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570523,0,CLASSE COMUM,14,ENSINO FUNDAMENTAL DE 9 ANOS,7,30,NaN
6,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570127,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,7,28,NaN
7,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570184,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,8,35,NaN
8,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570192,0,CLASSE COMUM,2,ENSINO MEDIO,1,35,NaN
9,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570218,0,CLASSE COMUM,2,ENSINO MEDIO,1,32,NaN
